## Dense Retrieval

In [1]:
!pip install faiss-cpu cohere rank_bm25 langchain langchain_community sentence-transformers
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import cohere

In [ ]:
COHERE_API_KEY = ""

In [4]:
co = cohere.Client(COHERE_API_KEY)

In [5]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

texts = text.split(".")
texts

['\nInterstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
 '\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 '\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 '\n\nBrothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
 '\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
 '\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 '\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',
 '\nInterstellar uses ext

In [6]:
texts = [t.strip(" \n") for t in texts]
texts

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
 'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
 'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',
 'Interstellar uses extensive practical a

In [7]:
response = co.embed(
    texts=texts,
    input_type="search_document"
).embeddings

In [8]:
response

[[0.21228027,
  -1.2519531,
  1.2949219,
  -2.3242188,
  1.2724609,
  -0.1875,
  2.078125,
  -1.171875,
  0.70751953,
  1.0800781,
  1.0634766,
  -0.70410156,
  -0.5996094,
  1.1875,
  1.0615234,
  -1.1923828,
  0.31689453,
  -0.50683594,
  0.6948242,
  0.3852539,
  0.5097656,
  -0.35351562,
  -0.34448242,
  0.81347656,
  -1.7763672,
  3.9023438,
  -0.36694336,
  -0.82910156,
  0.92041016,
  0.30273438,
  0.65527344,
  -0.46142578,
  0.5805664,
  0.6123047,
  0.48486328,
  -0.22143555,
  1.3916016,
  1.1113281,
  1.9238281,
  -1.0996094,
  -2.2773438,
  1.9296875,
  -1.8300781,
  -1.3261719,
  -0.37646484,
  -1.375,
  -0.7841797,
  0.26757812,
  0.47607422,
  -0.47265625,
  1.4492188,
  0.8486328,
  3.3847656,
  0.5126953,
  1.2246094,
  -0.6328125,
  0.39501953,
  0.8984375,
  1.7783203,
  -1.0927734,
  -0.88183594,
  -1.2451172,
  2.7460938,
  -0.5234375,
  0.07476807,
  1.8017578,
  -0.4206543,
  -1.40625,
  2.1523438,
  1.3525391,
  -5.0078125,
  0.17297363,
  -1.2353516,
  -0.9560

In [9]:
embeds = np.array(response)
embeds

array([[ 0.21228027, -1.2519531 ,  1.2949219 , ...,  1.5058594 ,
         0.88671875, -0.79052734],
       [ 2.3847656 , -0.5361328 ,  0.4560547 , ...,  0.8027344 ,
         0.1430664 , -0.21972656],
       [ 1.6035156 , -1.0830078 ,  1.0039062 , ...,  0.32128906,
        -1.4453125 , -0.3137207 ],
       ...,
       [ 2.0058594 , -0.42797852,  3.0253906 , ...,  0.84375   ,
        -1.9521484 , -0.23461914],
       [ 2.6191406 , -1.4667969 ,  1.8798828 , ...,  0.8066406 ,
        -0.19580078, -1.0117188 ],
       [ 0.5527344 , -2.6953125 ,  1.1621094 , ..., -1.7802734 ,
        -1.7197266 , -2.1289062 ]])

In [10]:
embeds.shape

(15, 4096)

In [11]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
print(index.is_trained)
index.add(np.float32(embeds))

True


In [12]:
def search(query, number_of_results=3):

    query_embed = co.embed(texts=[query], input_type="search_query").embeddings[0]

    distances, similar_item_ids = index.search(np.float32([query_embed]), number_of_results)

    texts_np = np.array(texts)
    results = pd.DataFrame(data={"texts": texts_np[similar_item_ids[0]],
                                 'distance': distances[0]})

    print(f"Query: {query} \nNearest Neighbours: ")

    return results

In [13]:
query = "How precise was the science ?"
results = search(query)
results

Query: How precise was the science ? 
Nearest Neighbours: 


,texts,distance
0,It has also received praise from many astronom...,10547.074219
1,Cinematographer Hoyte van Hoytema shot it on 3...,11502.488281
2,Interstellar uses extensive practical and mini...,11692.339844


## BM-25

In [15]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
    tokenized_doc = []

    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)

    return tokenized_doc

In [16]:
tokenized_corpus = []

for passage in tqdm(texts):
    tokenized_corpus.append(bm25_tokenizer(passage))

100%|██████████| 15/15 [00:00<00:00, 47197.72it/s]


In [17]:
tokenized_corpus

[['interstellar',
  '2014',
  'epic',
  'science',
  'fiction',
  'film',
  'co-written',
  'directed',
  'produced',
  'christopher',
  'nolan'],
 ['stars',
  'matthew',
  'mcconaughey',
  'anne',
  'hathaway',
  'jessica',
  'chastain',
  'irwin',
  'ellen',
  'burstyn',
  'matt',
  'damon',
  'michael',
  'caine'],
 ['set',
  'dystopian',
  'future',
  'humanity',
  'struggling',
  'survive',
  'film',
  'follows',
  'group',
  'astronauts',
  'travel',
  'wormhole',
  'near',
  'saturn',
  'search',
  'new',
  'home',
  'mankind'],
 ['brothers',
  'christopher',
  'jonathan',
  'nolan',
  'wrote',
  'screenplay',
  'origins',
  'script',
  'jonathan',
  'developed',
  '2007'],
 ['caltech',
  'theoretical',
  'physicist',
  '2017',
  'nobel',
  'laureate',
  'physics[4',
  'kip',
  'thorne',
  'executive',
  'producer',
  'acted',
  'scientific',
  'consultant',
  'wrote',
  'tie-in',
  'book',
  'science',
  'interstellar'],
 ['cinematographer',
  'hoyte',
  'van',
  'hoytema',
  '

In [19]:
bm25 = BM25Okapi(tokenized_corpus)

In [20]:
def keyword_search(query, top_k=3, num_candidates=15):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

## Reranker

In [21]:
query = "How precise was the science ?"
results = co.rerank(query=query, documents=texts, top_n=3, return_documents=True)
results

RerankResponse(id='c16699ff-6561-46f1-a311-a74ddafc2939', results=[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'), index=12, relevance_score=0.117785305), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'), index=10, relevance_score=0.036911618), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time'), index=13, relevance_score=0.0335294)], meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens

In [23]:
results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'), index=12, relevance_score=0.117785305),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'), index=10, relevance_score=0.036911618),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time'), index=13, relevance_score=0.0335294)]

In [24]:
for idx, result in enumerate(results.results):
    print(idx, result.relevance_score, result.document.text)
    print()

0 0.117785305 It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics

1 0.036911618 The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014

2 0.0335294 Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time



In [36]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

    #Add re-ranking
    docs = [texts[hit['corpus_id']] for hit in bm25_hits]

    print(f"\nTop-3 hits by rank-API ({len(bm25_hits)} BM25 hits re-ranked)")
    results = co.rerank(query=query, documents=docs, top_n=top_k, return_documents=True)
    for hit in results.results:
        print("\t{:.3f}\t{}".format(hit.relevance_score, hit.document.text.replace("\n", " ")))

In [37]:
keyword_and_reranking_search(query="How precise was the science ?")

Input question: How precise was the science ?
Top-3 lexical search (BM25) hits
	1.789	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	1.373	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.000	Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles

Top-3 hits by rank-API (10 BM25 hits re-ranked)
	0.118	It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics
	0.037	The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014
	0.034	Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time


## RAG

In [25]:
query = "income generated ?"

# Retrieval
results = search(query)
results

Query: income generated ? 
Nearest Neighbours: 


,texts,distance
0,The film had a worldwide gross over $677 milli...,14350.104492
1,Interstellar uses extensive practical and mini...,15117.125000
2,Cinematographer Hoyte van Hoytema shot it on 3...,15489.322266


In [26]:
docs_dict = [{'text': text} for text in results['texts']]
docs_dict

[{'text': 'The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'},
 {'text': 'Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects'},
 {'text': 'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm'}]

In [27]:
response = co.chat(
    message = query,
    documents = docs_dict
)
response

NonStreamedChatResponse(text='The film generated $677 million worldwide, or $773 million including subsequent re-releases.', generation_id='ec5143a4-5283-44b2-a238-c17529c65320', response_id='ddb3d10d-3305-4bfa-9d28-bec34bdc7e89', citations=[ChatCitation(start=19, end=41, text='$677 million worldwide', document_ids=['doc_0'], type='TEXT_CONTENT'), ChatCitation(start=46, end=92, text='$773 million including subsequent re-releases.', document_ids=['doc_0'], type='TEXT_CONTENT')], documents=[{'id': 'doc_0', 'text': 'The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'}], is_search_required=None, search_queries=None, search_results=None, finish_reason='COMPLETE', tool_calls=None, chat_history=[UserMessage(role='USER', message='income generated ?', tool_calls=None), ChatbotMessage(role='CHATBOT', message='The film generated $677 million worldwide, or $773 million including subsequent re-releases.'

In [28]:
response.text

'The film generated $677 million worldwide, or $773 million including subsequent re-releases.'

## RAG With Local Model

In [29]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-06-18 14:14:33--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.238.176.118, 18.238.176.129, 18.238.176.8, ...
Connecting to huggingface.co (huggingface.co)|18.238.176.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1750259673&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDI1OTY3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQ

In [30]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [31]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="thenlper/gte-small"
)

<ipython-input-31-2121023756>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
from langchain.vectorstores import FAISS

db = FAISS.from_texts(texts, embedding_model)

In [33]:
from langchain import PromptTemplate

template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context","question"]
)

In [34]:
from langchain.chains import RetrievalQA

rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [35]:
rag.invoke("Income generated ")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Income generated ',
 'result': ' The Income generated by the film was over $677 million worldwide, making it the tenth-highest grossing film of 2014. Additionally, after subsequent re-releases, the total income reached approximately $773 million.'}